# FinRL Complete Workflow Example

This notebook demonstrates the complete workflow from data acquisition to strategy training, stock selection, backtesting, and trade execution (Alpaca paper trading).

## Workflow Overview

1. **Environment Setup** - Clone repository and install dependencies
2. **Data Acquisition** - Fetch S&P 500 components and set research period
3. **Fundamental Data** - Use `data_fetcher` to get real fundamental data with forward-looking return labels
4. **ML Stock Selection** - Apply sector-rolling stock selection using `ml_strategy`
5. **Backtesting** - Run backtest using the `bt` library and output performance metrics
6. **Paper Trading** - Execute trades via Alpaca paper trading

## Prerequisites

- **FMP API Key**: Required for fundamental data (get free key at [Financial Modeling Prep](https://financialmodelingprep.com/))
- **Alpaca API Keys**: Required for paper trading (sign up at [Alpaca](https://alpaca.markets/))

---

## 1. Environment Setup

This section sets up the Google Colab environment by:
- Cloning the FinRL-Trading repository
- Installing required dependencies
- Configuring API credentials

In [ ]:
# Clone the FinRL-Trading repository
# This will download the complete codebase to your Colab environment

!git clone -b claude/create-finrl-colab-notebook-nnPau https://github.com/vrangayyan6/FinRL-Trading.git

# Change to the project directory
%cd FinRL-Trading

In [ ]:
# Install all required dependencies
# This may take a few minutes on first run

# Install dependencies from requirements.txt
!pip install -q -r requirements.txt

# Add the project directory to Python path
# This is required for importing src modules
import sys
import os

# Get the current working directory (should be FinRL-Trading after %cd)
project_root = os.getcwd()
if project_root not in sys.path:
    sys.path.insert(0, project_root)

# Also add src directory explicitly
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.insert(0, src_path)

print(f"Project root: {project_root}")
print(f"Python path configured for imports.")

# Verify key packages are installed
import importlib
required_packages = ['numpy', 'pandas', 'sklearn', 'lightgbm', 'yfinance', 'bt']
missing = []
for pkg in required_packages:
    try:
        importlib.import_module(pkg)
    except ImportError:
        missing.append(pkg)

if missing:
    print(f"Warning: Some packages may need manual installation: {missing}")
else:
    print("All core dependencies installed successfully!")

In [ ]:
# Configure API credentials
# IMPORTANT: Replace the placeholder values with your actual API keys

import os

# Financial Modeling Prep (FMP) API Key
# Get your free API key at: https://financialmodelingprep.com/developer/docs/
os.environ['FMP_API_KEY'] = 'your_fmp_api_key_here'  # <-- REPLACE THIS

# Alpaca Paper Trading API Keys (optional - only needed for trading section)
# Sign up at: https://alpaca.markets/
os.environ['ALPACA_API_KEY'] = 'your_alpaca_api_key_here'  # <-- REPLACE THIS
os.environ['ALPACA_SECRET_KEY'] = 'your_alpaca_secret_key_here'  # <-- REPLACE THIS
os.environ['ALPACA_BASE_URL'] = 'https://paper-api.alpaca.markets'  # Paper trading URL

print("Environment variables configured.")
print("NOTE: Make sure to replace placeholder API keys with your actual keys!")

In [ ]:
# Alternative: Use Google Colab Secrets (Recommended for security)
# Go to Colab menu: Runtime > Manage secrets
# Add your API keys there, then uncomment and run this cell

# from google.colab import userdata
#
# os.environ['FMP_API_KEY'] = userdata.get('FMP_API_KEY')
# os.environ['ALPACA_API_KEY'] = userdata.get('ALPACA_API_KEY')
# os.environ['ALPACA_SECRET_KEY'] = userdata.get('ALPACA_SECRET_KEY')
# os.environ['ALPACA_BASE_URL'] = 'https://paper-api.alpaca.markets'
#
# print("API keys loaded from Colab secrets.")

---

## 2. Data Source Initialization and Stock Universe

This section:
- Initializes the data manager
- Fetches S&P 500 component stocks
- Defines the research time period

In [ ]:
# Import data fetching utilities
from src.data.data_fetcher import get_data_manager, fetch_sp500_tickers

# Initialize the data manager
# This will use FMP (Financial Modeling Prep) as the data source
manager = get_data_manager()

# Display data source information
print("Data Manager Info:")
print(manager.get_source_info())

In [ ]:
# Fetch S&P 500 component stocks
# If the API call fails, a fallback list will be used

tickers = fetch_sp500_tickers()

print(f"Total S&P 500 components: {len(tickers)}")
print("\nSample tickers (first 15):")
print(tickers.head(15))

In [ ]:
# Define the research time period
# A longer period provides more data for rolling model training

start_date = '2015-10-15'  # Start date for historical data
end_date = '2025-10-15'    # End date (can be adjusted to current date)

print(f"Research Period: {start_date} to {end_date}")
print(f"Total years: ~{(2025 - 2015)} years of data")

---

## 3. Fetch Fundamental Data

This section fetches fundamental financial data for all S&P 500 stocks.

**Key Features:**
- `align_quarter_dates=True`: Aligns trading dates to approximately 2 months after earnings release
- Automatically calculates `y_return`: The next quarter's return (used as the prediction target)

**Note:** First-time fetching may take several minutes. Data is cached locally for faster subsequent runs.

In [ ]:
import pandas as pd
from src.data.data_fetcher import fetch_fundamental_data

# Fetch fundamental data for all tickers
# Note: For faster demo, you can limit tickers: tickers_small = tickers[:80]

print("Fetching fundamental data... This may take several minutes.")

fundamentals = fetch_fundamental_data(
    tickers,
    start_date,
    end_date,
    align_quarter_dates=True,  # Align dates to avoid look-ahead bias
)

print(f"\nFundamental data shape: {fundamentals.shape}")
print(f"Columns: {list(fundamentals.columns)}")

In [ ]:
# Validate the data contains required columns

# Check for y_return (prediction target - next quarter's return)
if 'y_return' not in fundamentals.columns:
    raise ValueError("ERROR: 'y_return' column missing. Check data_fetcher implementation.")

print("Data validation passed!")
print(f"\nSample of y_return distribution:")
print(fundamentals['y_return'].describe())

# Display sample data
print("\nSample data (first 5 rows):")
fundamentals.head()

In [ ]:
# Save the cleaned fundamental data for later use
import os

# Create data directory if it doesn't exist
os.makedirs('data', exist_ok=True)

# Save to CSV
clean_path = 'data/fundamentals.csv'
fundamentals.to_csv(clean_path, index=False)

print(f"Saved cleaned fundamentals to: {clean_path}")
print(f"File size: {os.path.getsize(clean_path) / 1024 / 1024:.2f} MB")

---

## 4. ML Stock Selection Strategy

This section uses machine learning to select stocks based on fundamental data.

**Strategy Options:**
- **Sector Neutral ML**: If sector data is available, performs sector-neutral stock selection
- **Basic ML**: Falls back to basic ML selection if no sector data

**Parameters:**
- `test_quarters`: Number of quarters for testing
- `top_quantile`: Select stocks in the top X% of predicted returns (0.9 = top 10%)
- `prediction_mode`: 'rolling' for walk-forward, 'single' for one-time prediction

In [ ]:
# Import ML strategy modules
from src.strategies.base_strategy import StrategyConfig
from src.strategies.ml_strategy import MLStockSelectionStrategy, SectorNeutralMLStrategy
import pandas as pd
import os

# Load the fundamental data
fundamentals = pd.read_csv('data/fundamentals.csv')
print(f"Loaded fundamentals: {fundamentals.shape[0]} rows, {fundamentals.shape[1]} columns")

# Prepare data dictionary for the strategy
data_dict = {'fundamentals': fundamentals}

In [ ]:
# Configure both strategy options

# Basic ML Stock Selection Strategy
base_config = StrategyConfig(
    name="ML Stock Selection",
    description="Machine learning based stock selection using fundamental data"
)
base_strategy = MLStockSelectionStrategy(base_config)

# Sector Neutral ML Strategy (used when sector data is available)
sector_config = StrategyConfig(
    name="Sector Neutral ML",
    description="Sector-neutral ML strategy for balanced portfolio construction"
)
sector_strategy = SectorNeutralMLStrategy(sector_config)

print("Strategies configured successfully.")

In [ ]:
# Run the stock selection strategy

# Strategy parameters
test_quarters = 4      # Number of quarters to test
top_quantile = 0.9     # Select top 10% of stocks (0.9 quantile)

# Check if sector column exists
sector_col_exists = ('sector' in fundamentals.columns) or ('gsector' in fundamentals.columns)

if sector_col_exists:
    # Use sector-neutral strategy for better diversification
    print("Using Sector Neutral ML Strategy...")
    result = sector_strategy.generate_weights(
        data_dict,
        test_quarters=test_quarters,
        top_quantile=top_quantile,
        prediction_mode='rolling'
    )
else:
    # Use basic ML strategy with single prediction mode
    print("Using Basic ML Stock Selection Strategy...")
    result = base_strategy.generate_weights(
        data=data_dict,
        prediction_mode='single',  # Single prediction (not rolling)
        test_quarters=test_quarters,
        top_quantile=top_quantile,
        weight_method='equal',     # Equal weight distribution
        confirm_mode='today',
        execution_date='2025-10-12'  # Order execution date
    )

# Extract weights from result
weights = result.weights.copy()
print(f"\nGenerated weights for {len(weights)} stock positions")

In [ ]:
# Save the generated weights to CSV

out_dir = 'data'
os.makedirs(out_dir, exist_ok=True)

if sector_col_exists:
    out_path = os.path.join(out_dir, 'ml_weights_sector.csv')
else:
    out_path = os.path.join(out_dir, 'ml_weights_today.csv')

weights.to_csv(out_path, index=False)

print(f"Saved portfolio weights to: {out_path}")
print(f"\nWeight distribution summary:")
print(weights.describe())

In [ ]:
# View the selected stocks and their weights

print("\nTop selected stocks by weight:")
print(weights.nlargest(10, 'weight') if 'weight' in weights.columns else weights.head(10))

---

## 5. Backtest the Strategy

This section backtests the ML-generated portfolio weights using the `bt` library.

**Steps:**
1. Load the portfolio weights
2. Fetch historical price data
3. Run the backtest simulation
4. Calculate and display performance metrics

In [ ]:
import pandas as pd
from src.backtest.backtest_engine import BacktestConfig, BacktestEngine
from src.data.data_fetcher import fetch_price_data

# Load the saved weights
weights_path = out_path  # From previous section
weights_raw = pd.read_csv(weights_path)

# Validate required columns
required_cols = {'date', 'gvkey', 'weight'}
assert required_cols.issubset(weights_raw.columns), f'Weight file must contain: {required_cols}'

# Parse and sort the data
weights_raw['date'] = pd.to_datetime(weights_raw['date'])
weights_raw['gvkey'] = weights_raw['gvkey'].astype(str)
weights_raw = weights_raw.sort_values(['date', 'gvkey'])

print(f"Loaded {len(weights_raw)} weight entries")
print(f"Date range: {weights_raw['date'].min()} to {weights_raw['date'].max()}")

In [ ]:
# Create weight matrix (rows: dates, columns: tickers)

weight_signals = (
    weights_raw
    .pivot_table(index='date', columns='gvkey', values='weight', aggfunc='sum')
    .fillna(0.0)
    .sort_index()
)

# Remove rows where all weights are zero
weight_signals = weight_signals.loc[(weight_signals.sum(axis=1) > 0.0)]

if len(weight_signals) == 0:
    raise ValueError('Weight matrix is empty - cannot run backtest')

print(f"Weight matrix shape: {weight_signals.shape}")
print(f"Unique rebalance dates: {len(weight_signals)}")
print(f"Unique tickers: {len(weight_signals.columns)}")

In [ ]:
# Configure the backtest

# Determine backtest period from weights
t_start = weight_signals.index.min().strftime('%Y-%m-%d')
t_end = weight_signals.index.max().strftime('%Y-%m-%d')

# Create backtest configuration
backtest_config = BacktestConfig(
    start_date=t_start,
    end_date=t_end,
    rebalance_freq='Q',        # Quarterly rebalancing
    initial_capital=1000000.0  # $1 million initial capital
)

print(f"Backtest Configuration:")
print(f"  Period: {t_start} to {t_end}")
print(f"  Rebalance Frequency: Quarterly")
print(f"  Initial Capital: ${backtest_config.initial_capital:,.0f}")

In [ ]:
# Fetch price data for backtest tickers

tickers_bt = weight_signals.columns.tolist()
print(f"Fetching price data for {len(tickers_bt)} tickers...")

prices_long = fetch_price_data(
    tickers_bt,
    backtest_config.start_date,
    backtest_config.end_date
)

if prices_long is not None and len(prices_long) > 0:
    print(f"Price data shape: {prices_long.shape}")
else:
    print("Warning: No price data fetched. Backtest may not work.")

In [ ]:
# Run the backtest
from pandas import DataFrame

# Initialize backtest engine
engine = BacktestEngine(backtest_config)

# Prepare price data for bt library format
price_data_bt = engine._prepare_price_data_for_bt(prices_long)

# Align weights with available price data
common_cols = [c for c in weight_signals.columns if c in price_data_bt.columns]
weight_signals = weight_signals[common_cols]

# Align weight dates to trading days
trading_index = price_data_bt.index
pos = trading_index.get_indexer(weight_signals.index, method='bfill')
mask = pos != -1
weight_signals = weight_signals.iloc[mask]
weight_signals.index = trading_index[pos[mask]]

# Clean and normalize weights for each date
aligned = []
for dt, row in weight_signals.iterrows():
    prices_today = price_data_bt.loc[dt, row.index]
    valid_cols = prices_today.dropna().index.tolist()
    if len(valid_cols) == 0:
        continue
    row_valid = row[valid_cols]
    s = row_valid.sum()
    if s <= 0:
        continue
    aligned.append((dt, (row_valid / s)))

if aligned:
    weight_signals = DataFrame({dt: vec for dt, vec in aligned}).T.sort_index()
else:
    raise ValueError('No valid weight rows after alignment')

# Final normalization (ensure weights sum to 1)
weight_signals = weight_signals.div(weight_signals.sum(axis=1), axis=0).fillna(0)

print(f"Final weight matrix: {weight_signals.shape}")

In [ ]:
# Execute backtest and display results

result = engine.run_backtest('ML Weights Strategy', prices_long, weight_signals)

print("="*60)
print("BACKTEST RESULTS")
print("="*60)
print(f"Backtest Period: {t_start} to {t_end}")
print(f"\nPortfolio Annualized Return: {result.annualized_return:.2%}")

# Display benchmark comparisons
if result.benchmark_annualized:
    print("\nBenchmark Comparisons:")
    for bm, ann in result.benchmark_annualized.items():
        print(f"  {bm} Annualized Return: {ann:.2%}")

# Display detailed metrics
print("\nDetailed Metrics:")
metrics_df = result.to_metrics_dataframe()
metrics_df

---

## 6. Alpaca Paper Trading

This section demonstrates live paper trading execution using Alpaca.

**Prerequisites:**
- Valid Alpaca API keys configured in Section 1
- Paper trading account enabled

**Note:** This section uses paper trading (simulated) and will NOT execute real trades or use real money.

In [ ]:
# Initialize the trade executor
import os
from src.trading.trade_executor import create_trade_executor_from_env
import pandas as pd

try:
    # Create trade executor from environment variables
    executor = create_trade_executor_from_env()
    print("Trade executor initialized successfully!")
    print(f"Available accounts: {executor.alpaca.get_available_accounts()}")

except Exception as e:
    print(f"Alpaca initialization failed: {e}")
    print("\nTo enable this section:")
    print("1. Sign up at https://alpaca.markets/")
    print("2. Get your API keys from the dashboard")
    print("3. Configure them in Section 1 of this notebook")

In [ ]:
# Prepare target weights for trading

try:
    # Load the weights file
    weights_path = 'data/ml_weights_today.csv'
    df = pd.read_csv(weights_path)
    df['date'] = pd.to_datetime(df['date']).dt.date

    # Get the latest date's weights
    latest_date = df['date'].max()
    df_latest = df[df['date'] == latest_date].copy()

    print(f"Using weights from: {latest_date}")
    print(f"Number of positions: {len(df_latest)}")

    # Normalize weights to sum to 1
    df_latest = df_latest.groupby('gvkey', as_index=False)['weight'].sum()
    df_latest = df_latest[df_latest['weight'] > 0]
    total_weight = df_latest['weight'].sum()

    if total_weight <= 0:
        raise ValueError('Total weight is zero - cannot create portfolio')

    df_latest['weight'] = df_latest['weight'] / total_weight

    # Convert to dictionary format for executor
    target_weights = {row['gvkey']: float(row['weight']) for _, row in df_latest.iterrows()}

    print(f"\nTarget portfolio ({len(target_weights)} positions):")
    for ticker, weight in sorted(target_weights.items(), key=lambda x: -x[1])[:10]:
        print(f"  {ticker}: {weight:.2%}")
    if len(target_weights) > 10:
        print(f"  ... and {len(target_weights) - 10} more positions")

except Exception as e:
    print(f"Error preparing weights: {e}")

In [ ]:
# Generate dry-run plan (preview without execution)

try:
    # Execute rebalance with dry_run=True (no actual orders)
    plan = executor.alpaca.execute_portfolio_rebalance(
        target_weights,
        account_name='default',
        dry_run=True  # Preview only
    )

    print("="*60)
    print("DRY-RUN TRADING PLAN")
    print("="*60)
    print(f"Market Open: {plan.get('market_open')}")
    print(f"Time in Force: {plan.get('used_time_in_force')}")
    print(f"\nPlanned Sell Orders: {len(plan.get('orders_plan', {}).get('sell', []))}")
    print(f"Planned Buy Orders:  {len(plan.get('orders_plan', {}).get('buy', []))}")

    # Show sample buy orders
    buy_orders = plan.get('orders_plan', {}).get('buy', [])[:5]
    if buy_orders:
        print("\nSample Buy Orders:")
        for order in buy_orders:
            print(f"  {order['symbol']}: {order['quantity']:.2f} shares ({order['order_type']})")

except Exception as e:
    print(f"Dry-run failed: {e}")

In [ ]:
# Execute paper trading (OPTIONAL - uncomment to run)
# WARNING: This will submit actual paper trading orders!

# Set this to True to enable order submission
ENABLE_TRADING = False

if ENABLE_TRADING:
    try:
        if plan.get('market_open'):
            # Market is open - submit regular orders
            submit = executor.alpaca.execute_portfolio_rebalance(
                target_weights,
                account_name='default'
            )
            print(f"Orders submitted during market hours.")
            print(f"Orders placed: {submit.get('orders_placed', 0)}")
        else:
            # Market is closed - use OPG (On Open) orders
            submit = executor.alpaca.execute_portfolio_rebalance(
                target_weights,
                account_name='default',
                market_closed_action='opg'  # Execute at market open
            )
            print(f"OPG orders submitted (will execute at market open).")
            print(f"Orders placed: {submit.get('orders_placed', 0)}")

    except Exception as e:
        print(f"Trade execution failed: {e}")
else:
    print("Trading is disabled. Set ENABLE_TRADING = True to submit orders.")
    print("Note: This uses paper trading - no real money involved.")

---

## 7. Performance Analysis

This section analyzes the paper trading account performance and compares it to benchmarks (SPY, QQQ).

In [ ]:
# Analyze Alpaca account performance
import logging
from datetime import datetime, timedelta, timezone
from src.trading.performance_analyzer import (
    create_alpaca_account_from_env, AlpacaManager,
    get_first_order_date, get_portfolio_history,
    get_benchmark_data, display_metrics_table, plot_performance
)

try:
    logging.basicConfig(level=logging.INFO)

    # Create Alpaca manager
    account = create_alpaca_account_from_env()
    manager = AlpacaManager([account])

    # Determine analysis period
    end_date = datetime.now(timezone.utc)
    first_order_date = get_first_order_date(manager)

    if first_order_date is None:
        raise RuntimeError('No order history found. Please execute some trades first.')

    # Set date range (from first order to now)
    start_date = first_order_date - timedelta(days=1)
    start_date_str = start_date.date().isoformat()
    end_date_str = (end_date + timedelta(days=1)).date().isoformat()

    print(f"Analysis Period: {start_date_str} to {end_date_str}")

    # Fetch portfolio and benchmark data
    portfolio_df = get_portfolio_history(manager, start_date, end_date)
    benchmark_df = get_benchmark_data(start_date_str, end_date_str)

    # Display metrics comparison
    display_metrics_table(portfolio_df, benchmark_df)

    # Plot performance chart
    plot_performance(portfolio_df, benchmark_df)

except Exception as e:
    print(f"Performance analysis failed: {e}")
    print("\nTo enable this section:")
    print("1. Configure Alpaca API keys")
    print("2. Execute some paper trades")
    print("3. Wait for order history to be available")

---

## 8. Cancel Open Orders (Optional)

Use this section to cancel all pending/open orders if needed.

In [ ]:
# Cancel all open orders (OPTIONAL)
from src.trading.performance_analyzer import create_alpaca_account_from_env, AlpacaManager

# Set this to True to enable order cancellation
ENABLE_CANCEL = False

if ENABLE_CANCEL:
    try:
        account = create_alpaca_account_from_env()
        manager = AlpacaManager([account])

        # Check current open orders
        open_orders = manager.get_orders(status='open', limit=200)
        print(f"Current open orders: {len(open_orders)}")

        if len(open_orders) > 0:
            # Cancel all open orders
            cancelled = manager.cancel_all_orders()
            print(f"Cancelled orders: {cancelled}")
        else:
            print("No open orders to cancel.")

    except Exception as e:
        print(f"Cancel orders failed: {e}")
else:
    print("Order cancellation is disabled. Set ENABLE_CANCEL = True to proceed.")

---

## Summary

This notebook demonstrated the complete FinRL workflow:

1. **Environment Setup**: Configured Colab with required dependencies and API keys
2. **Data Acquisition**: Fetched S&P 500 tickers and fundamental data
3. **ML Stock Selection**: Applied machine learning to select top-performing stocks
4. **Backtesting**: Evaluated strategy performance against historical data
5. **Paper Trading**: Executed (simulated) trades via Alpaca
6. **Performance Analysis**: Compared portfolio returns to benchmarks

## Next Steps

- Experiment with different `top_quantile` values (e.g., 0.8 for top 20%)
- Try rolling prediction mode for more robust results
- Add sector data for sector-neutral strategy
- Adjust rebalancing frequency in backtest config
- Monitor paper trading performance over time

## Resources

- [FinRL-Trading GitHub](https://github.com/AI4Finance-Foundation/FinRL-Trading)
- [Financial Modeling Prep API](https://financialmodelingprep.com/)
- [Alpaca Markets](https://alpaca.markets/)

---

*Created with FinRL-Trading - AI-Powered Stock Selection and Trading*